In [77]:

pos_normal = [int(i) for i in "77 515 6779622 6 91370 959685 0 9861".split(" ")]
pos_simple = [int(i) for i in "0 1 10 99 999".split(" ")]

i think the trick is that we shouldnt keep track of all stones together, because it will probably get VERY CROWDED.
we dont need the stone positions actually, and the stones dont interact with each other, so we just need to figure out, how many stones a certain stone yields after x blinks, where x is how many blinks remain after the stone was created 

In [ ]:

def blink_at(stone, times, verbose=False):
    if times == 0: 
        if verbose: print(stone)
        return 1 # if no more blinks, its just this stine
    elif stone == 0:
        return blink_at(1, times-1)
    elif (l:=len(stone_string:=str(stone)))%2 == 0: # even digits
        return blink_at(int(stone_string[:l//2]), times-1) + blink_at(int(stone_string[l//2:]), times-1)
    else:
        return blink_at(stone*2024, times-1)
    
sum(blink_at(n, 6) for n in [125,17])

22

In [79]:

sum([blink_at(n, 25) for n in pos_normal])

187738

In [80]:
#this doesnt work any more
# sum([blink_at(n, 75) for n in pos])

In [84]:
known_decay = {} # i cant keep track of anything?
# actually i can
# i should keep/build trails:
# if any new number with x remaining blinks is already in trails,
# it was with higher blinks
# XXX so i just need to follow the trail x steps to find how many stones were produced then XXX <-  this is where i initialy messed up
# 
def decay_products(stone): 
    if stone == 0:
        return [1]
    elif (l:=len(stone_string:=str(stone)))%2 == 0: # even digits
        return [int(stone_string[:l//2]),int(stone_string[l//2:])]
    else:
        return [stone*2024]

revisits = 0
def decay(stone, t, verbose=False):
    global revisits # for introspection
    if verbose: print(known_decay)

    if not stone in known_decay:
        if verbose: print("new decay product:", stone)
        known_decay[stone] = {0:1}
        
    if t in known_decay[stone]:
        revisits += 1
        return known_decay[stone][t]
    
    else: #t > known decay
        s = 0
        for product in decay_products(stone):
            known_decay[product][t-1] = decay(product, t-1)
            s+=known_decay[product][t-1]
        known_decay[stone][t] = s
        return s


decay(1,5)

known_decay, revisits


({1: {0: 1, 1: 1, 5: 7},
  2024: {0: 1, 4: 7},
  20: {0: 1, 3: 3},
  2: {0: 1, 2: 2},
  4048: {0: 1, 1: 2},
  40: {0: 1},
  48: {0: 1},
  0: {0: 1, 2: 1},
  24: {0: 1, 3: 4},
  4: {0: 1, 2: 2},
  8096: {0: 1, 1: 2},
  80: {0: 1},
  96: {0: 1}},
 6)

In [82]:
known_decay
pos = pos_normal
blinks = 76
for blink in range(1,blinks):
    for stone in pos:
        _= decay(stone, blink, verbose=False)
    print(revisits)
        
sum(known_decay[stone][75] for stone in pos),"revisits of nodes=", revisits

17
30
48
75
112
166
249
348
458
577
712
880
1092
1341
1631
1956
2330
2749
3215
3734
4319
4959
5664
6445
7304
8236
9246
10320
11463
12688
13978
15328
16746
18227
19760
21354
23023
24746
26538
28382
30260
32176
34125
36127
38175
40251
42355
44478
46615
48767
50935
53123
55328
57546
59772
62005
64244
66486
68732
70983
73240
75504
77772
80042
82312
84584
86858
89134
91413
93694
95977
98260
100543
102827
105111


(223767210249237, 'revisits of nodes=', 105111)

this was a nightmare to get right. i spent an entire afternoon.
ONe problem in my reasoning was that i assumed the trails to be continues, i.e. that if i knew the decay of X for time t, then i would also know decay of X at times t-1, which is not te case, because the decay products are all over the place.

or so i thought, but apparently it is continous, just not till the end for each decay product, wich makes sense...



In [83]:
revisits, known_decay

(105111,
 {1: {0: 1,
   1: 1,
   5: 7,
   2: 2,
   3: 4,
   4: 4,
   6: 14,
   7: 16,
   8: 20,
   9: 39,
   10: 62,
   11: 81,
   12: 110,
   13: 200,
   14: 328,
   15: 418,
   16: 667,
   17: 1059,
   18: 1546,
   19: 2377,
   20: 3572,
   21: 5602,
   22: 8268,
   23: 12343,
   24: 19778,
   25: 29165,
   26: 43726,
   27: 67724,
   28: 102131,
   29: 156451,
   30: 234511,
   31: 357632,
   32: 549949,
   33: 819967,
   34: 1258125,
   35: 1916299,
   36: 2886408,
   37: 4414216,
   38: 6669768,
   39: 10174278,
   40: 15458147,
   41: 23333796,
   42: 35712308,
   43: 54046805,
   44: 81997335,
   45: 125001266,
   46: 189148778,
   47: 288114305,
   48: 437102505,
   49: 663251546,
   50: 1010392024,
   51: 1529921658,
   52: 2327142660,
   53: 3537156082,
   54: 5362947711,
   55: 8161193535,
   56: 12380334344,
   57: 18810542532,
   58: 28593873985,
   59: 43369895096,
   60: 65961219181,
   61: 100139236592,
   62: 152044639077,
   63: 231153279853,
   64: 350788203951,
   6

In [ ]:
import functools

@functools.cache # <------------------------------- 😭😭😭😭😭😭😭
def blink_at(stone, times, verbose=False):
    if times == 0: 
        if verbose: print(stone)
        return 1 # if no more blinks, its just this stine
    elif stone == 0:
        return blink_at(1, times-1)
    elif (l:=len(stone_string:=str(stone)))%2 == 0: # even digits
        return blink_at(int(stone_string[:l//2]), times-1) + blink_at(int(stone_string[l//2:]), times-1)
    else:
        return blink_at(stone*2024, times-1)
    
sum(blink_at(n, 75) for n in pos_normal)

223767210249237

it couldve been so easy 😭😭😭😭😭😭😭😭😭😭😭

In [ ]:
from itertools import product


    

TypeError: 'int' object is not iterable

In [ ]:
print